In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import cv2
from tdmclient import ClientAsync, aw
from utils.camera import *
from utils.thymio import *
from utils.path import *
from utils.motion_control import *
from utils.buttons import*
from utils.drawings import *
from utils.color_thresholds import load_thresholds
###########################################################
# Parameters
###########################################################
COLOR_OBSTACLE = np.array([[80, 40, 0, 255, 70, 20]]) #BGR
COLOR_GOAL = np.array([0, 77, 0, 68, 255, 118])         #BGR
#COLOR_OBSTACLE = load_thresholds("color_obstacles.txt").reshape(1,-1) # run the notebook inside utils and save thresholds
#COLOR_GOAL = load_thresholds("color_goal.txt")

###########################################################
# Main Code
###########################################################
from tdmclient import ClientAsync, aw
client = ClientAsync()

async def main():

    node = await client.wait_for_node()
    aw(node.lock())
    
    print("Press forward button to start the program")
    beginning = False
    while(beginning == False) :
        beginning = await wait_for_start_button(node, client)
        time.sleep(0.3)
    cv2.destroyAllWindows()
    print("Starting the program")

    program = '''
        # Turn off LEDs during initialization
        call leds.temperature(0, 0)
        call leds.prox.v(0, 0)
        '''
    await node.compile(program)
    await node.run()
    
    # Camera initialization
    cam = camera_class(COLOR_OBSTACLE,COLOR_GOAL)
    while not cam.corners_found:
        cam = camera_class(COLOR_OBSTACLE, COLOR_GOAL)


    # Thymio initialization
    Thymio = Thymio_class(cam)
    step = 0
    
    while step<1000 :  
        
        step = step + 1
        
        # Update Image
        cam.get_image()
        cam.correct_perspective_aruco(get_matrix = False)
        
        # Thymio Position and motor
        Thymio.Thymio_position_aruco(cam.persp_image)
        Thymio.delta_time_update()

        if not Thymio.Thymio_detected:
            continue
        Thymio.xytheta_meas_hist = np.vstack((Thymio.xytheta_meas_hist, Thymio.xytheta_meas))


        #Check stop
        if(await check_stop_button(node, client)):
            aw(node.stop())
            aw(node.unlock())
            break
    cam.cam.release()
    aw(node.stop())
    aw(node.unlock())
    # cv2.destroyAllWindows()


# Run the main asynchronous function
while True:
    client.run_async_program(main)


In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
aw(node.lock())
#TEST START: aseba program to turn off leds
program = '''
# Turn off LEDs during initialization
    call leds.temperature(0, 0)
    call leds.prox.h(0, 0, 0, 0, 0, 0, 0, 0)
    call leds.prox.v(0, 0)
'''
await node.compile(program)
await node.run()
aw(node.stop())
aw(node.unlock())
# TEST END

In [ ]:
# Stop the program
aw(node.stop())
aw(node.unlock())

In [ ]:
cam.cam.release()

In [ ]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
aw(node.lock())

# Stop the program
aw(node.stop())
aw(node.unlock())

In [ ]:
###MARCHE PAS !!!!!!!!!        
        await node.set_variables({
            "leds.top": [0, 0, 0],
            "leds.circle": [0]*8,
            "leds.bottom.left": [0,0,0],
            "leds.bottom.right": [0,0,0],
            "leds.buttons": [0, 0, 0, 0],
            "leds.prox.h": [0]*8,
            "leds.prox.v": [0, 0],
            "leds.sound": [0],
            "leds.temperature": [0, 0]
        })

In [ ]:
import numpy as np
import cv2
from tdmclient import ClientAsync, aw
from camera_class import *
from thymio_class import *
from path import *
from motion_control import *
from buttons import*
from drawings import *

###########################################################
# Main Code
###########################################################
from tdmclient import ClientAsync, aw

client = ClientAsync()


async def main():

    node = await client.wait_for_node()
    aw(node.lock())
    await set_motors(node, 500, 500)
    start_time = time.time()
    if(time.time()>=(start_time+5)):
         await set_motors(node, 0, 0)  
         
    
# Run the main asynchronous function
client.run_async_program(main)
